# Importing Libraries

In [1]:
! pip install -U accelerate
! pip install -U transformers
! pip install torch datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import warnings
import numpy as np
import pandas as pd

import torch
import transformers

from datasets import Dataset
from datasets import load_metric

from tqdm import tqdm
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Importing The Model

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# Importing The Dataset

In [4]:
!gdown --id 1muNB7l4GcNvZPW34VFlnIliMR4Vk74SB

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1muNB7l4GcNvZPW34VFlnIliMR4Vk74SB
To: /content/ArzEn-MultiGenre_cleaned_data.xlsx
100% 1.37M/1.37M [00:00<00:00, 116MB/s]


In [5]:
# Load dataset into pandas DataFrame
data_path = "/content/ArzEn-MultiGenre_cleaned_data.xlsx"
data = pd.read_excel(data_path)

data.head()


,Unnamed: 0,egyption_Text,english_Text,category,sub_category
0,0,الأمير الصغير,The little prince,Novels,the-little-prince
1,1,أنطوان دي سانت إكزوبيري,Antoine De Saint-Exuper,Novels,the-little-prince
2,2,الفصل الأول,Chapter 1,Novels,the-little-prince
3,3,في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...,Once when I was six years old I saw a magnific...,Novels,the-little-prince
4,4,وكانت عباره عن تعبان من نوع البوا بيبلع فهد.,It was a picture of a boa constrictor in the a...,Novels,the-little-prince


In [6]:
data.isnull().sum()

Unnamed: 0       0
egyption_Text    0
english_Text     0
category         0
sub_category     0
dtype: int64

In [7]:
data = data[['egyption_Text' , 'english_Text']]
data.head()

,egyption_Text,english_Text
0,الأمير الصغير,The little prince
1,أنطوان دي سانت إكزوبيري,Antoine De Saint-Exuper
2,الفصل الأول,Chapter 1
3,في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...,Once when I was six years old I saw a magnific...
4,وكانت عباره عن تعبان من نوع البوا بيبلع فهد.,It was a picture of a boa constrictor in the a...


# Data Splitting

In [8]:
train, validation_test = train_test_split(data, test_size=0.2, random_state=42)
validation, test = train_test_split(validation_test, test_size=0.5, random_state=42)

In [9]:
train_data = Dataset.from_pandas(train)
validation_data = Dataset.from_pandas(validation)
test_data = Dataset.from_pandas(test)

train_data , validation_data , test_data

(Dataset({
     features: ['egyption_Text', 'english_Text', '__index_level_0__'],
     num_rows: 13367
 }),
 Dataset({
     features: ['egyption_Text', 'english_Text', '__index_level_0__'],
     num_rows: 1671
 }),
 Dataset({
     features: ['egyption_Text', 'english_Text', '__index_level_0__'],
     num_rows: 1671
 }))

# Data Tokenziation

In [10]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex for ex in examples["egyption_Text"]]
    targets = [ex for ex in examples["english_Text"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [11]:
tokenized_train = train_data.map(
    preprocess_function,
    batched=True,
    remove_columns=train_data.column_names,
)

Map:   0%|          | 0/13367 [00:00<?, ? examples/s]

In [12]:
tokenized_validation = validation_data.map(
    preprocess_function,
    batched=True,
    remove_columns=validation_data.column_names,
)

Map:   0%|          | 0/1671 [00:00<?, ? examples/s]

In [35]:

tokenized_test = test_data.map(
    preprocess_function,
    batched=True,
    remove_columns=test_data.column_names,
)

Map:   0%|          | 0/1671 [00:00<?, ? examples/s]

# Setting up The GPU

In [13]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! Training on GPU...")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Training on CPU...")
    device = torch.device("cpu")

CUDA is available! Training on GPU...


# Setting up the Evaluation Metric  

In [15]:
from datasets import load_dataset, load_metric

# Load BLEU metric
metric = load_metric("bleu")

# Define function to compute BLEU score
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return metric.compute(predictions=decoded_preds, references=decoded_labels)


<ipython-input-15-871ea4d3b2a8>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("bleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


# Importing the Model

In [45]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

training_args = Seq2SeqTrainingArguments(
    output_dir="./output_dir",  # specify the output directory where model checkpoints and logs will be saved
    per_device_train_batch_size=16,
    per_device_eval_batch_size = 8,
    num_train_epochs=5,
    logging_dir="./logs",
    learning_rate=3e-5,
)

In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [27]:
trainer = Seq2SeqTrainer(
    model=model.to(device),
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,2.441200
1000,2.193100
1500,1.937400
2000,1.811600
2500,1.726400
3000,1.575800
3500,1.546500
4000,1.490600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=4180, training_loss=1.8260060214540035, metrics={'train_runtime': 596.1556, 'train_samples_per_second': 112.11, 'train_steps_per_second': 7.012, 'total_flos': 695500962988032.0, 'train_loss': 1.8260060214540035, 'epoch': 5.0})

# Saving The Model

In [29]:
trainer.save_model('Finetuned Model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}


# Model Evaluation

In [38]:
print(type(tokenized_test))

<class 'datasets.arrow_dataset.Dataset'>


In [39]:
tokenized_test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1671
})

In [ ]:
test_dataset = Dataset.from_dict({
    "input_ids": tokenized_test["input_ids"],
    "attention_mask": tokenized_test["attention_mask"],
    "labels": tokenized_test["labels"],
})


trainer.evaluate(test_dataset=test_dataset,)


print("Evaluation results:", evaluation_results)

# Model Testing

In [32]:
from transformers import pipeline
model_path = "/content/Finetuned Model"
translator = pipeline("translation", model=model_path, tokenizer="Helsinki-NLP/opus-mt-ar-en")

arabic_sentences = [
    "السلام عليكم , ازيك؟ عامل ايه؟",
]

translated_sentences = translator(arabic_sentences, max_length=50, return_text=True)

for original, translation in zip(arabic_sentences, translated_sentences):
    print("Arabic:", original)
    print("English:", translation)
    print()

Arabic: السلام عليكم , ازيك؟ عامل ايه؟
English: {'translation_text': 'Hello, how are you?'}

